## PBL(1) Project 를 위한 실습(연습)

1. https://agtechresearch.pythonanywhere.com/ 에 접속하여 회원가입해 주세요. (비밀번호는 단순하게 만드는 것을 권장. 예: 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.

In [ ]:
# project = "housingprice"  # 수정하지 마세요
# username = ""  # 회원가입 시 사용한 이메일아이디 (예시. abc@hello.com)
# password = ""  # 비밀번호

리더보드 제출을 위한 기본 설정: 아래 코드를 실행해주세요.


In [ ]:
# import os
# import urllib.request

# if not os.path.exists("competition.py"):
#     url = "https://raw.githubusercontent.com/agtechresearch/LectureMLbasic/refs/heads/main/competition/competition.py"
#     filename = "competition.py"
#     urllib.request.urlretrieve(url, filename)

In [ ]:
!wget https://github.com/agtechresearch/LectureMLbasic/raw/refs/heads/main/housingprice/dataset.zip

!unzip dataset.zip -d data

아래 코드를 실행하여 데이터를 다운로드 받습니다: 3개의 csv 파일이 data 폴더에 다운로드됨

 * dataset.csv: 과거 주택매매 데이터 -> 학습에 사용할 데이터셋
 * problem.csv: 현재 A사가 매매를 고려하고 있는 130건의 주택정보 -> ML 모델에 의하여 예측을 수행하여야 할 데이터셋
 * submission.csv: 리더보드 서버 제출을 위한 파일 형식


----------

### 아래는 랜덤포레스트를 사용하여 집값 예측 모델을 만들고, 코랩환경에서 결과를 리더보드에 제출하는 간단한 샘플 코드입니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# Data 경로 설정
DATA_DIR = "data"

In [ ]:
# 학습에 사용할 과거 주택매매 data set 로드 (dataset.csv)
dataset = pd.read_csv(os.path.join(DATA_DIR, "dataset.csv"))

# problem set 로드 (problem.csv)
problemset = pd.read_csv(os.path.join(DATA_DIR, "problem.csv"))

In [ ]:
# 변수가 24개가 있으므로, 모든 변수를 출력하려면 다음과 같이 설정해야 함
pd.options.display.max_columns = 30

dataset  # 학습에 사용할 과거 주택매매 데이터셋 확인: 1340건의 과거 주택매매 데이터셋

### **<데이터 구성>**

* LotArea : 평방피트 단위의 부지 크기  
* Street : 부동산으로의 도로(street) 접근 유형    
* LotConfig : 지역 구성  

* OverallQual : 전체적인 재료 및 마감 등급  
* OverallCond : 집의 전체적인 상태 등급  
* YearBuilt : 원래 공사일  
* YearRemodAdd : 리모델링 날짜 (리모델링 혹은 추가가 없었던 경우 공사일과 동일)  

* TotalBsmtSF : 지하 총 평방피트  

* 1stFlrSF : 1층 평방피트  
* 2ndFlrSF : 2층 평방피트  

* GrLivArea : 지상 거실 면적 평방 피트  

* FullBath : 지하층 위의 전체 욕실  
* HalfBath : 지하층 위의 반 욕실  
* BedroomAbvGr : 지하층 위의 침실 수 (지하 침실은 포함하지 않음)  
* KitchenAbvGr : 지하층 위의 주방 수  
  
* TotRmsAbvGrd : 지하층 위의 모든 방 (욕실 제외)  
  
* FirePlace : 벽난로 수  

* GarageType : 차고 위치  
* GarageYrBlt : 차고 건설 연도  
  
* GarageCars : 차량 수용 가능 차고 크기  
* GarageArea : 차고 평방피트  

* YrSold : 연 판매  
* SalePrice : 판매 가격

In [ ]:
# problem set 확인: 130건의 문제 데이터셋 (SalePrice를 예측해야 함)
problemset

## 데이터 전처리 및 모델 학습

In [ ]:
# data 와 problem 데이터들을 합쳐서 하나의 데이터로 만들어줍니다 --> 아래의 데이터 전처리 후 다시 분리할 예정
all_data = pd.concat([dataset, problemset], ignore_index=True)

# select_dtypes 함수를 사용하여 데이터 타입이 object(문자형 데이터)인 열을 제외
# select_dtypes 함수는 열에 포함된 데이터들을 type 기준으로 인덱싱 할 수 있도록 해줌
# select_dtypes(include=None, exclude=None) 형태를 가지며, include에 넣은값을 포함하고 exclude에 넣은 값을 제외한 columns(열)을 DaraFrame 형태로 반환함
all_data = all_data.select_dtypes(exclude="object")

# 결측치를 0으로 대체
all_data = all_data.fillna(0)

# Id 열을 제거
all_data = all_data.drop(["Id"], axis=1)

In [ ]:
# 학습 데이터와 문제 데이터로 다시금 분리
train_data = all_data[: len(dataset)]
problem_data = all_data[len(dataset):]

In [ ]:
# 학습 데이터의 SalePrice 열을 제외한 나머지 열을 X로 지정, SalePrice 열을 Y로 지정
X = train_data.drop("SalePrice", axis=1)
Y = train_data["SalePrice"]

In [ ]:
# 모델 학습을 위해 학습 데이터를 80%의 학습 데이터(train)와 20%의 검증 데이터(test)로 나눔
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# 랜덤 포레스트 모델을 사용하여 학습
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(x_train, y_train)

In [ ]:
# train 데이터와 test 데이터에 대한 예측값을 구하고 RMSE 값을 계산
from sklearn.metrics import mean_squared_error

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

print("Train RMSE :", mean_squared_error(y_train, train_pred) ** 0.5)
print("Test RMSE :", mean_squared_error(y_test, test_pred) ** 0.5)

RMSE: 평균 제곱근 오차

## Problem set 문제에 대한 집값예측 및 리더보드 결과 제출

- 아래 제출 프로세스가 느리다고 중지 후 다시 코드를 여러차례 재실행하는 경우 패널티가 발생할 수 있습니다. (제출 과정에서 제출 횟수 이슈 발생 가능: 하루 최대 200회 까지 가능)
- 제출에 성공할 경우, "제출에 성공하였습니다"의 메세지와 함께 제출 결과 RMSE 가 화면에 출력됩니다.
- 제출결과는 또한 [대회 페이지(리더보드 서버)](https://agtechresearch.pythonanywhere.com/competitions/housingprice/)의 `리더보드` 와 `제출` 탭에서 확인할 수 있습니다.


In [ ]:
# 전처리 과정 중에 SalePrice 가 0으로 채워져 있기 때문에, problem_data 에서 SalePrice를 다시 제거
problem_data = problem_data.drop("SalePrice", axis=1)
# 문제 데이터(problem data)에 대한 예측값을 구함
problem_pred = model.predict(problem_data)

In [ ]:
# 리더보드 서버 제출을 위한 파일 생성
submission = pd.read_csv(os.path.join(DATA_DIR, "submission.csv"))
submission["SalePrice"] = problem_pred

# 예측 결과 화면에 출력 후 제출
display(submission)
competition.submit(project, username, password, submission)